## ID : 312441

In [75]:
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import torch as pt 
from torch.utils.data import Dataset, DataLoader , Subset, ConcatDataset, ChainDataset
from torch.nn import functional as F
from torch import nn
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [76]:
# This function is responsible for getting our data loaders. Here we accept two parameters which are 
# flags to whether implement Data augmention, normalization or not. After adding/skipping augmentation, normalization
# we move towards next stop of loading our train and test dataset and return the output back to main function.
def generateDataLoader(aug = None, normlze = None):
    c_modules = []

    if aug:
        c_modules = c_modules + [transforms.RandomHorizontalFlip(), transforms.RandomResizedCrop(32, scale = (0.8, 1))]

    c_modules = c_modules + [transforms.ToTensor()]
    ts_c_modules = [transforms.ToTensor()]

    if normlze:
        c_modules = c_modules + [transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
        ts_c_modules = ts_c_modules + [transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]

    tr_dataset = CIFAR10(root = ".", train = True, download = True, transform = transforms.Compose(c_modules))
    ts_dataset = CIFAR10(root = ".", train = False, download = True, transform = transforms.Compose(ts_c_modules))

    if (aug is None) & (normlze is None):
        tr_dataset = pt.utils.data.Subset(tr_dataset, np.arange(len(tr_dataset)//2) * 2)

    tr_dl = DataLoader(tr_dataset, batch_size = 1000, shuffle = True)
    ts_dl = DataLoader(ts_dataset, batch_size = 1000, shuffle = False)

    return tr_dl, ts_dl

In [77]:
# This class is where we define our neural network following the architecture given in assignment.
# for conv1 32x32x3 -> 30x30x8, for conv2 15x15x8 -> 13x13x16, for conv3 13x13x16 goes to 10x10x32.
# added the fully connected layer with ReLU. Here also we have flags for adding batch_normalization,
# dropout layer depending on the experiment we are running.
class CustomNN(nn.Module):
    def __init__(self, dout = None, b_norm = None):
        super(CustomNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, (3,3))  
        self.conv2 = nn.Conv2d(8, 16, (3,3)) 
        self.conv3 = nn.Conv2d(16, 32, (4,4))

        self.fc1   = nn.Linear(32 * 5 * 5, 256)
        self.fc2   = nn.Linear(256, 256)
        self.fc3   = nn.Linear(256, 128)
        self.fc4   = nn.Linear(128, 10)

        self.dout = dout
        self.b_norm = b_norm

        if self.b_norm is not None:
            self.batch_norm1 = nn.BatchNorm1d(num_features=256)
            self.batch_norm2 = nn.BatchNorm1d(num_features=256)
            self.batch_norm3 = nn.BatchNorm1d(num_features=128)

    def forward(self, tx):
        tx = F.max_pool2d(pt.relu(self.conv1(tx)), (2, 2))
        tx = pt.relu(self.conv2(tx))
        tx = F.max_pool2d(pt.relu(self.conv3(tx)), (2, 2))
        tx = tx.view(-1, 32 * 5 * 5)
        tx = pt.relu(self.fc1(tx))
    
        if self.b_norm is not None:
            tx = self.batch_norm1(tx)  
        if self.dout is not None:
            tx = nn.Dropout(0.2)(tx)
        tx = pt.relu(self.fc2(tx))
        
        if self.b_norm is not None:
            tx = self.batch_norm2(tx)
        if self.dout is not None:
            tx = nn.Dropout(0.2)(tx)
        tx = pt.relu(self.fc3(tx))

        if self.b_norm is not None:
            tx = self.batch_norm3(tx)
        if self.dout is not None:
            tx = nn.Dropout(0.2)(tx)
        tx = F.softmax(self.fc4(tx),dim=1)

        return tx

In [78]:
# This is the function which is being triggered from main function, with parameters like summarywriter,
# max_iterations and exp_num. From exp_num we are finding out exactly which exercise we are working on
# a brief guide for exp_num is given with main code. So in this function we create our convolution network
# with experiment requirment, get the dataloaders and call our training function.
def execute_exp(exp_num, sum_output, max_iter = 30):
    config = exp_setting[exp_num]

    c_net = CustomNN(dout = config.get('dout', None), b_norm = config.get('b_norm', None))
    c_net.to('cpu')

    if config.get('regu', None) == 'l2':
        l2 = 10e-5
        l1 = None
    elif config.get('regu', None) == 'l1':
        l1 = 10e-7
        l2 = 0
    else:
        l1 = None
        l2 = 0

    if config.get('lr', None) is not None:
        l_r = config.get('lr', None)
    else:
        l_r = 0.001

    all_optim = {'adam':pt.optim.Adam(c_net.parameters(), lr = l_r, weight_decay = l2),
                        'sgd':pt.optim.SGD(c_net.parameters(), lr = l_r, weight_decay = l2, momentum = 0.9)}
    
    optim = config.get('optim', 'adam')
    c_optim = all_optim[optim]

    tr_dl, ts_dl = generateDataLoader(aug = config.get('aug', None), normlze = config.get('norml', None))
    loss_func = nn.CrossEntropyLoss()
    cnet_tr(tr_dl, ts_dl, c_net, loss_func, c_optim, config.get('exp_desc'), l1 = l1, max_iter = max_iter)

In [79]:
# This is the function where training is happening invoked by execute_exp(). We are simply computing the 
# prediction and losses applying regulatization if required in experiment then backpropagating the losses
# applying the optimizer meanwhile logging the data too for displaying at tensorboard later.
# Ref: https://shonit2096.medium.com/cnn-on-cifar10-data-set-using-pytorch-34be87e09844#:~:text=The%20goal%20is%20to%20apply,Learning%20and%20Computer%20Vision%20algorithms.
def cnet_tr(tr_dl, ts_dl, c_net, loss_func, c_optim, exp_desc, l1, max_iter = 30): 
    tr_size = len(tr_dl.dataset)
    for iter in tqdm(range(max_iter), leave = True):
        avg_loss, accu = 0, 0
        for batch, (X, y) in tqdm(enumerate(tr_dl), leave = False):
            c_optim.zero_grad()
            X, y = X.to('cpu'), y.to('cpu')
            pred = c_net(X)
            loss_val = loss_func(pred, y)
            if l1:
                regL1 = pt.tensor(0., requires_grad = True)
                for name, param in c_net.named_parameters():
                    if 'weight' in name:
                        regL1 = regL1 + pt.sum(pt.abs(param))

                loss_val = loss_val + (l1 * regL1)

            loss_val.backward()
            c_optim.step()
            tr_accu = (pred.argmax(1) == y).type(pt.float).sum().item()*100/X.size(0)
            tr_loss = loss_val.item()
            iter_chk = iter * len(tr_dl) + batch

            if (iter_chk % 25 == 0) or (iter_chk == 0):
                test_size = len(ts_dl.dataset)
                test_loss, test_acc = 0, 0
                with pt.no_grad():
                    for X, y in tqdm(ts_dl, leave = False):
                        X, y = X.to('cpu'), y.to('cpu')
                        pred = c_net(X)
                        test_loss = test_loss + loss_func(pred, y).item() * X.size(0)
                        test_acc = test_acc + (pred.argmax(1) == y).type(pt.float).sum().item()

                test_loss = test_loss / test_size
                test_acc = test_acc / test_size
                test_acc = test_acc * 100
            else:
                pass

            sum_output.add_scalars(f'Loss_{exp_desc}', {"Train":tr_loss,"Test":test_loss}, iter_chk)
            sum_output.add_scalars(f'Accuracy_{exp_desc}', {"Train":tr_accu,"Test":test_acc}, iter_chk)
            
            sum_output.add_scalars(f'Train_Accuracy', {f"{exp_desc}":tr_accu}, iter_chk)
            sum_output.add_scalars(f'Test_Accuracy', {f"{exp_desc}":test_acc}, iter_chk)

            sum_output.add_scalars(f'Train_loss', {f"{exp_desc}":tr_loss}, iter_chk)
            sum_output.add_scalars(f'Test_loss', {f"{exp_desc}":test_loss}, iter_chk)

            accu = accu + tr_accu * X.size(0)

            avg_loss = avg_loss + tr_loss * X.size(0)

In [80]:
# I have made a dictionary with different setting as per requirement from each exercise to run
# the experiments with specific setting using my generic functions.

exp_setting = {1: {}, 2: {'aug': True}, 3: {'norml': True}, 4: {'aug': True, 'norml': True}, 5: {'aug': True, 'norml': True, 'dropout': True}, 6: {'aug': True, 'norml': True, 'regu': "l1"}, 7: {'aug': True, 'norml': True, 'regu': "l2"}, 8: {'aug': True, 'norml': True, 'dropout': True, 'regu': "l2"}, 9:  {'norml': True, 'optim': 'adam', 'lr': 0.0001}, 10: {'norml': True, 'optim': 'adam', 'lr': 0.001}, 11: {'norml': True, 'optim': 'adam', 'lr': 0.01}, 12: {'norml': True, 'optim': 'adam', 'lr': 0.1}, 13: {'norml': True, 'optim': 'sgd', 'lr': 0.5}, 14: {'norml': True, 'optim': 'sgd', 'lr': 0.001}, 15: {'norml': True, 'optim': 'sgd', 'lr': 0.01}, 16: {'norml': True, 'optim': 'sgd', 'lr': 0.1}, 17: {'norml': True,'dropout': True, 'batch_norm': True}}

exp_desc = {1: 'Baseline',
                2: 'Augmentation Applied',
                3: 'Normalization Applied',
                4: 'Augmentation + Normalization Applied',

                5: 'Dropout Applied',
                6: 'L1 Regularization Applied',
                7: 'L2 Regularization Applied',
                8: 'Dropout + L2 Regularization Applied',

                9 :  'Adam Opt with LR 0.0001',
                10 : 'Adam Opt with LR 0.001',
                11 : 'Adam Opt with LR 0.01',
                12 : 'Adam Opt with LR 0.1',
                13 : 'SGD with LR 0.5',
                14 : 'SGD with LR 0.001',
                15 : 'SGD with LR 0.01',
                16 : 'SGD with LR 0.1',
                17 : 'Batch Normalization Applied',}

for i in range(1, 18):
    exp_setting[i].update({'exp_desc': exp_desc[i]})
    print(i, exp_setting[i])


1 {'exp_desc': 'Baseline'}
2 {'aug': True, 'exp_desc': 'Augmentation Applied'}
3 {'norml': True, 'exp_desc': 'Normalization Applied'}
4 {'aug': True, 'norml': True, 'exp_desc': 'Augmentation + Normalization Applied'}
5 {'aug': True, 'norml': True, 'dropout': True, 'exp_desc': 'Dropout Applied'}
6 {'aug': True, 'norml': True, 'regu': 'l1', 'exp_desc': 'L1 Regularization Applied'}
7 {'aug': True, 'norml': True, 'regu': 'l2', 'exp_desc': 'L2 Regularization Applied'}
8 {'aug': True, 'norml': True, 'dropout': True, 'regu': 'l2', 'exp_desc': 'Dropout + L2 Regularization Applied'}
9 {'norml': True, 'optim': 'adam', 'lr': 0.0001, 'exp_desc': 'Adam Opt with LR 0.0001'}
10 {'norml': True, 'optim': 'adam', 'lr': 0.001, 'exp_desc': 'Adam Opt with LR 0.001'}
11 {'norml': True, 'optim': 'adam', 'lr': 0.01, 'exp_desc': 'Adam Opt with LR 0.01'}
12 {'norml': True, 'optim': 'adam', 'lr': 0.1, 'exp_desc': 'Adam Opt with LR 0.1'}
13 {'norml': True, 'optim': 'sgd', 'lr': 0.5, 'exp_desc': 'SGD with LR 0.5'}

In [81]:
# This is the main function where we are executing our main function changing the experiment_num param

# A short guide for experiment numbers.
# 1-4 (Exercise 1) {1 : baseline, 2: augmentation, 3: normalization,4: augmentation + normalization}
# 5-8 (Exercise 2) {5: dropout, 6: l1 regularization, 7: L2 regularization, 8: dropout + L2 regularization}
# 9-12 (Exercise 3) {9: Adam opt lr=0.0001, 10: Adam opt lr=0.001, 11: Adam opt lr=0.01, 12: Adam opt lr=0.1}
# 13-16 (Exercise 3) {13: sgd opt lr= 0.5, 14: sgd opt lr= 0.001, 15: sgd opt lr= 0.01, 16: sgd opt lr= 0.1} [have to change optimizer in execute_exp()] 
# 17 Batch normalization (Good difference as seen in the report)
exp_no = 4
logs = 'run/experiment_cifar_exp' + str(exp_no)
sum_output = SummaryWriter(logs)

c_net = CustomNN()

print('Executing experiment : ', str(exp_no))
execute_exp(exp_no, sum_output, max_iter = max_iter)

Executing experiment :  4
Files already downloaded and verified
Files already downloaded and verified


  0%|          | 0/30 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [82]:
%tensorboard --logdir run/experiment_cifar_exp4 --host localhost --port 8112